# AWS Lambda Functions

Many web developers are familiar with AWS’s S3 buckets for storage and EC2 instances for running applications but AWS has come a long way beyond S3 and EC2. There are higher level services introduced in recent past that greatly simplify cloud services, at a cheaper cost, and requiring little maintenance.

Many a times there would be large spikes of activity on a website or application and far less activity during rest of the time. In order to serve the high traffic of data during these spikes we will be running many EC2 instances, but during lulls its not economical to run those EC2 instances. Lambda functions prove to be the optimal solution to solve this problem.

In this notebook, we will deploy Lambda functions on AWS.

## AWS Lambda functions

An AWS Lambda function is a collection of code with a single entry point, or handler, and can be written in either Node.js, Python (2.7 or higher), or Java (8). They take a single input. A Lambda function, generally, can be thought of as a simple function taking a single input and performing some transformation. They are not restricted to simple operations. They can be quite complex, since libraries can be uploaded to the function. The Lambda function works by being configured to fire in response to some event, such as data added to a AWS Kinesis stream, or files uploaded to an S3 bucket. Behind the scenes, Amazon stores the Lambda function code and configuration on S3 and when an event fires, it creates (or possibly reuses) a container, and passes the event data to the function handler.

With this architecture AWS handles the management of the function and gives users three main advantages:

### Serverless (simple)

Lambda functions don’t require the maintenance of a server. The resources a lambda function uses is set only by specifying it’s memory usage, from 128 MB to 1536 MB. CPU speed is scaled as memory goes up and the memory allocation can be changed at any time.

### Auto-scaling (smart)

There is no scaling to enable or to configure. Lambda functions spawn as necessary to keep up with the pace of events, although bandwidth may be restricted depending on other services accessed (e.g., bandwidth to a Kinesis stream is based on number of shards in the stream).

### Cost-effective (cheap)

Compared to an EC2 instance which is up 24/7 and incurring costs, you are only charged based on how much the function runs, with no costs when it is not running. Furthermore, a function that has been allocated 512 GB of memory only costs 3 cents for every hour of computation. For a function that takes 1 second, those 3 cents can buy you over 3000 invocations of your function. Even for large scale operations, that are performing millions of functions a month, the cost will still typically be less than $20.



### Deploying Lambda functions with Python and boto3

The AWS console is a great way to get started on Lambda functions. It steps you through the process of creating one and includes templates for different languages. However the AWS console lacks an automated way to add new functions and update existing ones. The AWS CLI (command line interface) can be automated, however when the Lambda functions are complex (for instance, when they require event source mappings and IAM roles and policies) doing these actions in Python scripts using boto3 is easier.

Using a simple example, I’ll demonstrate how to use boto3 to create a function with an associated role and policies, and how to update that code via Python scripts. We’ll start with a simplified version of one of the AWS Lambda example templates in the file lambda.py.

The example is taken from [this post](https://developmentseed.org/blog/2016/03/08/aws-lambda-functions/)

In [2]:
import os
import zipfile

stream_name = 'twitter_stream2'


zf = zipfile.ZipFile("lambda.zip", "w")
zf.write('lambda.py')
zf.close()

Below python script will create a Kinesis stream, an IAM access role and create a Lambda function with the zip file as input, then finally map the stream to the Lambda function.

In [1]:
import time
import json
import boto3

kinesis = boto3.client('kinesis')
iam = boto3.client('iam')
lamb = boto3.client('lambda')

def create_stream(name):
    """ Create kinesis stream, and wait until it is active """
    if name not in [f for f in kinesis.list_streams()['StreamNames']]:
        print('Creating Kinesis stream %s' % (name))
        kinesis.create_stream(StreamName=name, ShardCount=1)
    else:
        print('Kinesis stream %s exists' % (name))
    while kinesis.describe_stream(StreamName=name)['StreamDescription']['StreamStatus'] == 'CREATING':
        time.sleep(2)
    return kinesis.describe_stream(StreamName=name)['StreamDescription']


def create_role(name, policies=None):
    """ Create a role with an optional inline policy """
    policydoc = {
        "Version": "2012-10-17",
        "Statement": [
            {"Effect": "Allow", "Principal": {"Service": ["lambda.amazonaws.com"]}, "Action": ["sts:AssumeRole"]},
        ]
    }
    roles = [r['RoleName'] for r in iam.list_roles()['Roles']]
    if name in roles:
        print('IAM role %s exists' % (name))
        role = iam.get_role(RoleName=name)['Role']
    else:
        print('Creating IAM role %s' % (name))
        role = iam.create_role(RoleName=name, AssumeRolePolicyDocument=json.dumps(policydoc))['Role']

    # attach managed policy
    if policies is not None:
        for p in policies:
            iam.attach_role_policy(RoleName=role['RoleName'], PolicyArn=p)
    return role


def create_function(name, zfile, lsize=512, timeout=120, update=False):
    """ Create, or update if exists, lambda function """
    role = create_role(name + '_lambda', policies=['arn:aws:iam::aws:policy/service-role/AWSLambdaKinesisExecutionRole'])
    print("role:",role)
    
    with open(zfile, 'rb') as zipfile:
        if name in [f['FunctionName'] for f in lamb.list_functions()['Functions']]:
            if update:
                print('Updating %s lambda function code' % (name))
                return lamb.update_function_code(FunctionName=name, ZipFile=zipfile.read())
            else:
                print('Lambda function %s exists' % (name))
                for f in funcs:
                    if f['FunctionName'] == name:
                        lfunc = f
        else:
            print('Creating %s lambda function' % (name))
            lfunc = lamb.create_function(
                FunctionName=name,
                Runtime='python3.6',
                Role=role['Arn'],
                Handler='lambda.lambda_handler',
                Description='Example lambda function to ingest a Kinesis stream',
                Timeout=timeout,
                MemorySize=lsize,
                Publish=True,
                Code={'ZipFile': zipfile.read()},
            )
        lfunc['Role'] = role
        return lfunc

def create_mapping(name, stream):
    """ add a mapping to a stream """
    print("checking event source mappings")
    sources = lamb.list_event_source_mappings(FunctionName=name,
                                           EventSourceArn=stream['StreamARN'])['EventSourceMappings']
    for s in sources:
        print(s)
        
    if stream['StreamARN'] not in [s['EventSourceArn'] for s in sources]:
        print("stream not present in event sources")
        source = lamb.create_event_source_mapping(FunctionName=name, EventSourceArn=stream['StreamARN'],
                                      StartingPosition='TRIM_HORIZON')
    else:
        for s in sources:
            source = s
    return source


# create kinesis stream
stream = create_stream(stream_name)

# Create a lambda function
lfunc = create_function(stream_name, 'lambda.zip', update=True)

NameError: name 'stream_name' is not defined

In [ ]:
response = lamb.list_functions(
    FunctionVersion='ALL'
)
if lfunc['FunctionName'] in [s['FunctionName'] for s in response['Functions']]:
    print('function is created')

In [ ]:
# add mapping to kinesis stream
create_mapping(stream_name, stream)

In [1]:
# aws kinesis put-record --stream-name alonzo --data "{'0': 'the', '1': 'lambda', '2': 'calculus'}"


response = kinesis.put_record(
    StreamName='twitter',
    Data="{'0': 'the', '1': 'lambda', '2': 'calculus'}",
    PartitionKey='1'
#     ExplicitHashKey='string',
#     SequenceNumberForOrdering='string'
)

NameError: name 'kinesis' is not defined

In [ ]:
from TwitterAPI import TwitterAPI
import boto3
import json
import twitterCreds

## twitter credentials

consumer_key = twitterCreds.consumer_key
consumer_secret = twitterCreds.consumer_secret
access_token_key = twitterCreds.access_token_key
access_token_secret = twitterCreds.access_token_secret

api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

# kinesis = boto3.client('kinesis')

# r = api.request('statuses/filter', {'track':'modi'})
# # print(r)
# tweets = []
# count = 0
# for item in r:
#     jsonItem = json.dumps(item)
#     print(jsonItem)
#     tweets.append({'Data':jsonItem[0], 'PartitionKey':"filler"})
#     count += 1
#     if count == 100:
#         print(count)
#         kinesis.put_records(StreamName="twitter", Records=tweets)
# #         print(tweets[1])
#         count = 0
#         tweets = []
        
r = api.request('statuses/filter', {'track': 'modi'})

# Writes new tweets into Kinesis
for item in r:
    if 'text' in item:
        kinesis.put_record(StreamName='twitter', Data=json.dumps(item), PartitionKey=item['user']['screen_name'])
        print item['text']

In [ ]:
type(tweets[0]['Data'])

We should have an IAM role named sai_lambda, and a Kinesis stream and lambda function named sai. Print the response variable.

In [ ]:
response

The JSON response has the ShardId. You can go to CloudWatch in AWS web interface and check the logs to see if the function worked. Under Logs there should be an `"/aws/lambda/sai"` log group. Within that are different log streams. New log streams will be created periodically, but a single log stream may hold the logs from more than a single invocation of the function. You should see the output of the event and the decoded JSON we went into the stream.

While Lambda functions cost nothing when not running, it is typically used in a system where there are other resources which do cost something. Kinesis streams are charged per hour per shard. This example only creates a single shard so sai stream will cost $0.36 a day. Make sure to delete the resource once done playing with it.

In [ ]:
response = kinesis.delete_stream(
    StreamName='sai'
)

In [ ]:
response